In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import re

# Defining Model

In [2]:
num_classes = 2
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(num_classes, activation='softmax'))

# not using the first layer for training
model.layers[0].trainable = False

2022-04-20 00:09:29.508352: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-20 00:09:29.521560: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


94773248/94765736 [==============================] - 3s 0us/step


# Compile Model

In [3]:
model.compile(optimizer='sgd', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

# Image augmentation for preprocessing image

In [4]:
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                              horizontal_flip = True, 
                                              width_shift_range = 0.1,
                                              height_shift_range = 0.1)
            
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

# Take image directly from director and apply augmentation

In [5]:
image_size = 224

train_generator = data_generator_with_aug.flow_from_directory(
                                        directory='/Users/sherapgyaltsen/Documents/Face-mask-detection/train',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical')
validation_generator = data_generator_no_aug.flow_from_directory(
                                        directory='/Users/sherapgyaltsen/Documents/Face-mask-detection/val',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

Found 36 images belonging to 2 classes.
Found 36 images belonging to 2 classes.


# Fit the model

In [11]:
fit_stats = model.fit_generator(train_generator,
                                       steps_per_epoch=60,
                                       epochs = 10,
                                       validation_data=validation_generator,
                                       validation_steps=1)

/Users/sherapgyaltsen/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
60/60 [==============================] - 9s 121ms/step - loss: 0.3105 - accuracy: 0.8611 - val_loss: 0.0034 - val_accuracy: 1.0000


# Testing with test image

In [12]:
test_generator = data_generator_no_aug.flow_from_directory(
    directory='/Users/sherapgyaltsen/Documents/Face-mask-detection/test',
    target_size=(image_size, image_size),
    batch_size= 10,
    class_mode=None,
    shuffle=False
)

pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
cl = np.round(pred)

filenames=test_generator.filenames


real_class = []
for file in filenames:
  if re.search("with_mask", file):
    real_class.append(1.0)
  else:
    real_class.append(0.0)

predicted_class = cl[:,0]

results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "pred_class":predicted_class, "real_class":real_class})

Found 14 images belonging to 2 classes.


/Users/sherapgyaltsen/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2/2 [==============================] - 2s 459ms/step


In [13]:
results

,file,pr,pred_class,real_class
0,with_mask/opencv_frame_0.png,0.991802,1.0,1.0
1,with_mask/opencv_frame_1.png,0.998883,1.0,1.0
2,with_mask/opencv_frame_2.png,0.999575,1.0,1.0
3,with_mask/opencv_frame_3.png,0.999243,1.0,1.0
4,with_mask/opencv_frame_4.png,0.999897,1.0,1.0
5,with_mask/opencv_frame_5.png,0.999828,1.0,1.0
6,with_mask/opencv_frame_6.png,0.999850,1.0,1.0
7,without_mask/img1.png,0.001084,0.0,0.0
8,without_mask/img2.png,0.002584,0.0,0.0
9,without_mask/img3.png,0.004708,0.0,0.0


In [14]:
res = sum(1 for x,y in zip(real_class,predicted_class) if x == y) / len(real_class)
print("Accuracy :", res)

Accuracy : 1.0


# Save Model

In [15]:
model.save('maskDetectionModel_1.h5')